# Limpieza de datos

## ETL

Gran parte del tiempo de un científico de datos se invierte en la limpieza de datos. En este notebook veremos un ejemplo de cómo limpiar datos utilizando la librería `pandas` de Python.

ETL es un acrónimo que significa Extract, Transform, Load. Es un proceso que se utiliza para extraer datos de una fuente, transformarlos en un formato que sea adecuado para el análisis y cargarlos en una base de datos o algún otro sistema de almacenamiento.

## Datos
Los datos que usaremos para esta limpieza y nuestro siguiente análisis son datos de incidencia delictiva en nuestro país.

La iniciativa de datos abiertos del gobierno de México nos proporciona datos de incidencia delictiva desde 2015 hasta la fecha. Los datos se actualizan todos los meses y se pueden descargar desde el siguiente enlace: https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva

---

Como podemos ver en el portal, se proporcionan los datos tanto a nivel estatal como a nivel municipal. En este caso, utilizaremos los datos a nivel estatal.

Descarguemos los datos y guardemos el archivo CSV en la carpeta data con el nombre `datos_delitos.csv`.

---

Ahora leamos el archivo CSV y veamos cómo se ven los datos.

Primero que nada, importemos pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/datos_delitos.csv')

Aquí tenemos un error muy común que suele ocurrir cuando un archivo se guarda en una computadora con cierto "encoding". 

Un encoding es una tabla que relaciona un número con un carácter. Por ejemplo, en la tabla ASCII, el número 65 corresponde a la letra "A".

Si el archivo que estamos leyendo fue guardado con un encoding distinto al que pandas espera, nos arrojará un error. 

Para solucionar esto, podemos utilizar el parámetro `encoding` de la función `pd.read_csv()` y especificar el encoding correcto.

Podemos probar con `ISO-8859-1` o `latin1` que son los más comunes.

# Cargamos el archivo de incidencia_delictiva.csv

```python
import pandas as pd

df = pd.read_csv('data/datos_delitos.csv', encoding='ISO-8859-1')
df.head()
```

In [ ]:
df = pd.read_csv('data/datos_delitos.csv', encoding='ISO-8859-1')

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
df.info()

Haremos las siguientes limpiezas:
* Transformar los nombres de las columnas para que no tengan caracteres especiales y estén siempre en minúsculas
* Convertir el dataset a un formato de datos "largo"

¿Qué significa formato largo?
Como podemos ver aquí tenemos los meses como si fueran variables. Es decir, los meses son columnas en este set de datos. Lo ideal sería contar con una columna "fecha" que junte "Año" y "Mes".

Sure! Here's a function that cleans the column names of a pandas DataFrame:


In [ ]:
def clean_column_names(df):
    columnas_limnas = []
    for col in df.columns:
        col = col.lower().replace(" ", "_").replace("ñ", "ni").replace(".", "")
        columnas_limnas.append(col)
    
    df.columns = columnas_limnas
    
    return df


In [ ]:
df = clean_column_names(df)

In [ ]:
df.head(3)

In [ ]:
df = df[['anio', 'clave_ent', 'entidad', 'cve_municipio', 'municipio', 'tipo_de_delito', 'subtipo_de_delito', 'modalidad','enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']]

Ahora, usaremos el método `melt` para convertir las columnas a observaciones

In [ ]:
datos_long = df.melt(id_vars=['anio', 'clave_ent', 'entidad', 'cve_municipio', 'municipio', 'tipo_de_delito', 'subtipo_de_delito', 'modalidad'], var_name='nombre_mes', value_name='frecuencia')

In [ ]:
datos_long.head()

In [ ]:
datos_long.info()

In [ ]:
# Further group the dataframe. I only want "tipo_de_delito" and i dont care about the levels subtipe and modalidad. I want a single record per municipio, month and tipo_de_delito

datos_long = datos_long.groupby(['anio', 'clave_ent', 'entidad', 'cve_municipio', 'municipio', 'tipo_de_delito', 'nombre_mes'])['frecuencia'].sum().reset_index()

Mostremos todos los estados y su respectiva clave

In [ ]:
datos_long[['clave_ent', 'entidad']].drop_duplicates()

Ahora Veamos los municipios de un estado en particular. Por ejemplo Nuevo León

In [ ]:
datos_long[datos_long['clave_ent'] == 19][['entidad','cve_municipio', 'municipio']].drop_duplicates()

Ahora veamos todos los homicidios del año 2023 del municipio San Pedro García Garza

In [ ]:
datos_long[(datos_long['anio'] == 2023) & (datos_long['cve_municipio'] == 19019) & (datos_long['tipo_de_delito'] == "Homicidio")]

Ya que tenemos muestros datos bien estructurados, los podemos guardar en nuestra computadora. Los guardaremos con el nombre "delitos.csv"

In [ ]:
datos_long.to_csv('data/delitos.csv', index=False)